<a href="https://colab.research.google.com/github/fleanend/TorchGlocalK/blob/main/Original_Glocal_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from time import time
from scipy.sparse import csc_matrix
import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error
torch.manual_seed(1284)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Loader Function

In [2]:
def load_data_from_csv(path='./', delimiter=',', max_user = None):
    df = pd.read_csv(path, index_col=0)
    print(df.columns)
    rows_with_nan = df[df.isna().any(axis=1)]
    print(rows_with_nan)
    df = df[['userId', 'movieId', 'rating']]

    # print( df['movieId'])
    if max_user:
        df = df.loc[df['userId'] <= max_user]
    def discrete(column):
        sorted_values = sorted(df[column].unique(), reverse=True)
        def custom_rank(value):
            return sorted_values.index(value) + 1
        return custom_rank

    # df['userId'] = df['userId'].apply(discrete('userId'))
    # df['movieId'] = df['movieId'].apply(discrete('movieId'))
    # df.to_csv('discreted_link_ratings.csv')
    n_u = df['userId'].nunique()
    n_m = df['movieId'].nunique()
    # Chia tập train từ tập dữ liệu
    train, df_temp = train_test_split(df, test_size=0.4, shuffle=True, random_state=42)

    # Chia tập test và tập validation từ tập dữ liệu còn lại
    test, val = train_test_split(df_temp, test_size=0.5, shuffle=True, random_state=42)
    print(train.shape, val.shape, test.shape, n_m, n_u)
    # print(train.iloc[0])
    # print(df.iloc[0])
    train_r = np.zeros((n_m, n_u))
    val_r = np.zeros((n_m, n_u))
    test_r = np.zeros((n_m, n_u))

    n_train = train.shape[0]  # num of training ratings
    n_val = val.shape[0]  # num of valid ratings
    n_test = test.shape[0]  # num of test ratings

    for i in range(n_train):
        # print(train.iloc[i,1]-1, train.iloc[i,0]-1)
        train_r[train.iloc[i,1]-1, train.iloc[i,0]-1] = train.iloc[i,2]

    for i in range(n_val):
        # print(train.iloc[i,1]-1, train.iloc[i,0]-1)
        val_r[val.iloc[i,1]-1, val.iloc[i,0]-1] = val.iloc[i,2]

    for i in range(n_test):
        test_r[test.iloc[i,1]-1, test.iloc[i,0]-1] = test.iloc[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    val_m = np.greater(val_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of valid ratings: {}'.format(n_val))
    print('num of test ratings: {}'.format(n_test))
    return n_m, n_u, train_r, train_m, val_r, val_m, test_r, test_m


# Load Data

In [3]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
data_path = '../processed_data_for_matrix_completion/discreted_link_ratings_p0.csv'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._


In [4]:
# Data Load
path = data_path
n_m, n_u, train_r, train_m, val_r, val_m, test_r, test_m = load_data_from_csv(path=path, delimiter='\t')


Index(['userId', 'movieId', 'rating'], dtype='object')
Empty DataFrame
Columns: [userId, movieId, rating]
Index: []
(117680, 3) (39227, 3) (39227, 3) 14363 1999
data matrix loaded
num of users: 1999
num of movies: 14363
num of training ratings: 117680
num of valid ratings: 39227
num of test ratings: 39227


In [5]:
# Common hyperparameter settings
n_hid = 500 # size of hidden layers
n_dim = 5 # inner AE embedding size
n_layers = 2 # number of hidden layers
gk_size = 3 # width=height of kernel for convolution

# Hyperparameters to tune for specific case
max_epoch_p = 500 # max number of epochs for pretraining
max_epoch_f = 1000 # max number of epochs for finetuning
patience_p = 5 # number of consecutive rounds of early stopping condition before actual stop for pretraining
patience_f = 10 # and finetuning
tol_p = 1e-4 # minimum threshold for the difference between consecutive values of train rmse, used for early stopping, for pretraining
tol_f = 1e-5 # and finetuning
lambda_2 = 20. # regularisation of number or parameters
lambda_s = 0.006 # regularisation of sparsity of the final matrix
dot_scale = 1 # dot product weight for global kernel

# Network Functions

In [6]:
def local_kernel(u, v):
    dist = torch.norm(u - v, p=2, dim=2)
    hat = torch.clamp(1. - dist**2, min=0.)
    return hat

class KernelLayer(nn.Module):
    def __init__(self, n_in, n_hid, n_dim, lambda_s, lambda_2, activation=nn.Sigmoid()):
      super().__init__()
      self.W = nn.Parameter(torch.randn(n_in, n_hid))
      self.u = nn.Parameter(torch.randn(n_in, 1, n_dim))
      self.v = nn.Parameter(torch.randn(1, n_hid, n_dim))
      self.b = nn.Parameter(torch.randn(n_hid))

      self.lambda_s = lambda_s
      self.lambda_2 = lambda_2

      nn.init.xavier_uniform_(self.W, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.xavier_uniform_(self.u, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.xavier_uniform_(self.v, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.zeros_(self.b)
      self.activation = activation

    def forward(self, x):
      w_hat = local_kernel(self.u, self.v) # (n_in, n_hid)
    
      sparse_reg = torch.nn.functional.mse_loss(w_hat, torch.zeros_like(w_hat))
      sparse_reg_term = self.lambda_s * sparse_reg
      
      l2_reg = torch.nn.functional.mse_loss(self.W, torch.zeros_like(self.W))
      l2_reg_term = self.lambda_2 * l2_reg

      W_eff = self.W * w_hat  # Local kernelised weight matrix
      y = torch.matmul(x, W_eff) + self.b # (m, n_hid)
      y = self.activation(y) 

      return y, sparse_reg_term + l2_reg_term

class KernelNet(nn.Module):
    def __init__(self, n_u, n_hid, n_dim, n_layers, lambda_s, lambda_2):
      super().__init__()
      layers = []
      for i in range(n_layers):
        if i == 0:
          layers.append(KernelLayer(n_u, n_hid, n_dim, lambda_s, lambda_2))
        else:
          layers.append(KernelLayer(n_hid, n_hid, n_dim, lambda_s, lambda_2))
      layers.append(KernelLayer(n_hid, n_u, n_dim, lambda_s, lambda_2, activation=nn.Identity())) # output(m, n_u)
      self.layers = nn.ModuleList(layers)
      self.dropout = nn.Dropout(0.5)

    def forward(self, x):
      total_reg = None
      for i, layer in enumerate(self.layers):
        x, reg = layer(x)
        if i < len(self.layers)-1:
          x = self.dropout(x)
        if total_reg is None:
          total_reg = reg
        else:
          total_reg += reg
      return x, total_reg

In [7]:
class CompleteNet(nn.Module):
    def __init__(self, kernel_net, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale):
      super().__init__()
      self.gk_size = gk_size
      self.dot_scale = dot_scale
      self.local_kernel_net = kernel_net # first stage
      self.global_kernel_net = KernelNet(n_u, n_hid, n_dim, n_layers, lambda_s, lambda_2) # last stage
      self.conv_kernel = torch.nn.Parameter(torch.randn(n_m, gk_size**2) * 0.1)
      nn.init.xavier_uniform_(self.conv_kernel, gain=torch.nn.init.calculate_gain("relu"))
      
    # reference to figure 1 in the paper
    def forward(self, train_r):
      x, _ = self.local_kernel_net(train_r) # pretrain with Local Kernel-base
      gk = self.global_kernel(x, self.gk_size, self.dot_scale) # Create global kernel (GK) (equation 4, 5)
      x = self.global_conv(train_r, gk) # convolution operation 𝑅ˆ = 𝑅 ⊗ 𝐺𝐾 equation 6
      x, global_reg_loss = self.global_kernel_net(x) # fine tuning with Global Kernel-based Matrix
      return x, global_reg_loss

    def global_kernel(self, input, gk_size, dot_scale):
      avg_pooling = torch.mean(input, dim=1)  # Item (axis=1) based average pooling
      avg_pooling = avg_pooling.view(1, -1)

      gk = torch.matmul(avg_pooling, self.conv_kernel) * dot_scale  # Scaled dot product
      gk = gk.view(1, 1, gk_size, gk_size)

      return gk

    def global_conv(self, input, W):
      input = input.unsqueeze(0).unsqueeze(0)
      conv2d = nn.LeakyReLU()(F.conv2d(input, W, stride=1, padding=1))
      return conv2d.squeeze(0).squeeze(0)

class Loss(nn.Module):
    def forward(self, pred_p, reg_loss, train_m, train_r):
      # L2 loss
      diff = train_m * (train_r - pred_p)
      sqE = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff))
      loss_p = sqE + reg_loss
      return loss_p

# Network Instantiation

## Pre-training

In [8]:
model = KernelNet(n_u, n_hid, n_dim, n_layers, lambda_s, lambda_2).double().to(device)

## Fine-tuning

In [9]:
complete_model = CompleteNet(model, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale).double().to(device)

# Evaluation code

In [10]:
def dcg_k(score_label, k):
    dcg, i = 0., 0
    for s in score_label:
        if i < k:
            dcg += (2**s[1]-1) / np.log2(2+i)
            i += 1
    return dcg

In [11]:
def ndcg_k(y_hat, y, k):
    score_label = np.stack([y_hat, y], axis=1).tolist()
    score_label = sorted(score_label, key=lambda d:d[0], reverse=True)
    score_label_ = sorted(score_label, key=lambda d:d[1], reverse=True)
    norm, i = 0., 0
    for s in score_label_:
        if i < k:
            norm += (2**s[1]-1) / np.log2(2+i)
            i += 1
    dcg = dcg_k(score_label, k)
    return dcg / norm

In [12]:
def call_ndcg(y_hat, y):
    ndcg_sum, num = 0, 0
    y_hat, y = y_hat.T, y.T
    n_users = y.shape[0]

    for i in range(n_users):
        y_hat_i = y_hat[i][np.where(y[i])]
        y_i = y[i][np.where(y[i])]

        if y_i.shape[0] < 2:
            continue

        ndcg_sum += ndcg_k(y_hat_i, y_i, y_i.shape[0])  # user-wise calculation
        num += 1

    return ndcg_sum / num

# Training and Test Loop

In [13]:
best_rmse_ep, best_mae_ep, best_ndcg_ep = 0, 0, 0
best_rmse, best_mae, best_ndcg = float("inf"), float("inf"), 0

time_cumulative = 0
tic = time()

# Pre-Training
optimizer = torch.optim.AdamW(complete_model.local_kernel_net.parameters(), lr=0.001)

def closure():
  optimizer.zero_grad()
  x = torch.Tensor(train_r).double().to(device)
  m = torch.Tensor(train_m).double().to(device)
  complete_model.local_kernel_net.train()
  pred, reg = complete_model.local_kernel_net(x)
  loss = Loss().to(device)(pred, reg, m, x)
  loss.backward()
  return loss

last_rmse = np.inf
counter = 0

for i in range(max_epoch_p):
  optimizer.step(closure)
  complete_model.local_kernel_net.eval()
  t = time() - tic
  time_cumulative += t

  pre, _ = model(torch.Tensor(train_r).double().to(device))
  
  pre = pre.float().cpu().detach().numpy()
  
  error = (val_m * (np.clip(pre, 1., 5.) - val_r) ** 2).sum() / val_m.sum()  # val error
  val_rmse = np.sqrt(error)

  error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
  train_rmse = np.sqrt(error_train)

  if last_rmse-train_rmse < tol_p:
    counter += 1
  else:
    counter = 0

  last_rmse = train_rmse

  if patience_p == counter:
    print('.-^-._' * 12)
    print('PRE-TRAINING')
    print('Epoch:', i+1, 'val rmse:', val_rmse, 'train rmse:', train_rmse)
    print('Time:', t, 'seconds')
    print('Time cumulative:', time_cumulative, 'seconds')
    print('.-^-._' * 12)
    break


  if i % 50 != 0:
    continue
  print('.-^-._' * 12)
  print('PRE-TRAINING')
  print('Epoch:', i, 'val rmse:', val_rmse, 'train rmse:', train_rmse)
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)

.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 0 val rmse: 2.6424843061107093 train rmse: 2.6414038224585976
Time: 4.226903200149536 seconds
Time cumulative: 4.226903200149536 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 28 val rmse: 1.1277923592997117 train rmse: 1.103697659503925
Time: 91.32204341888428 seconds
Time cumulative: 1317.8092114925385 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._


In [14]:
# Fine-Tuning
optimizer = torch.optim.AdamW(complete_model.parameters(), lr=0.001)

def closure():
  optimizer.zero_grad()
  x = torch.Tensor(train_r).double().to(device)
  m = torch.Tensor(train_m).double().to(device)
  complete_model.train()
  pred, reg = complete_model(x)
  loss = Loss().to(device)(pred, reg, m, x)
  loss.backward()
  return loss

last_rmse = np.inf
counter = 0

for i in range(max_epoch_f):
  optimizer.step(closure)
  complete_model.eval()
  t = time() - tic
  time_cumulative += t

  pre, _ = complete_model(torch.Tensor(train_r).double().to(device))
  
  pre = pre.float().cpu().detach().numpy()

  error = (val_m * (np.clip(pre, 1., 5.) - val_r) ** 2).sum() / val_m.sum()  # val error
  val_rmse = np.sqrt(error)

  error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
  train_rmse = np.sqrt(error_train)

  val_mae = (val_m * np.abs(np.clip(pre, 1., 5.) - val_r)).sum() / val_m.sum()
  train_mae = (train_m * np.abs(np.clip(pre, 1., 5.) - train_r)).sum() / train_m.sum()

  val_ndcg = call_ndcg(np.clip(pre, 1., 5.), val_r)
  train_ndcg = call_ndcg(np.clip(pre, 1., 5.), train_r)

  if val_rmse < best_rmse:
      best_rmse = val_rmse
      best_rmse_ep = i+1

  if val_mae < best_mae:
      best_mae = val_mae
      torch.save(complete_model.state_dict(), 'Glocal-K_best.pth')
      best_mae_ep = i+1

  if best_ndcg < val_ndcg:
      best_ndcg = val_ndcg
      best_ndcg_ep = i+1

  if last_rmse-train_rmse < tol_f:
    counter += 1
  else:
    counter = 0

  last_rmse = train_rmse

  if patience_f == counter:
    print('.-^-._' * 12)
    print('FINE-TUNING')
    print('Epoch:', i+1, 'val rmse:', val_rmse, 'val mae:', val_mae, 'val ndcg:', val_ndcg)
    print('Epoch:', i+1, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
    print('Time:', t, 'seconds')
    print('Time cumulative:', time_cumulative, 'seconds')
    print('.-^-._' * 12)
    break


  if i % 50 != 0:
    continue

  print('.-^-._' * 12)
  print('FINE-TUNING')
  print('Epoch:', i, 'val rmse:', val_rmse, 'val mae:', val_mae, 'val ndcg:', val_ndcg)
  print('Epoch:', i, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)

.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 0 val rmse: 2.63881348395579 val mae: 2.4330083762695467 val ndcg: 0.8494356085642428
Epoch: 0 train rmse: 2.63797145434049 train mae: 2.430917803688232 train ndcg: 0.8318347764663317
Time: 97.7194721698761 seconds
Time cumulative: 1415.5286836624146 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 50 val rmse: 1.016921407175779 val mae: 0.7968755306391725 val ndcg: 0.885078461644091
Epoch: 50 train rmse: 0.9880486897616867 train mae: 0.7780053150406863 train ndcg: 0.8749428296174129
Time: 559.75257396698 seconds
Time cumulative: 18072.58071756363 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 100 val rmse: 0.9533205891607958 val mae: 0.7456952926327

In [15]:
# Final result
print('Epoch:', best_rmse_ep, ' best rmse:', best_rmse)
print('Epoch:', best_mae_ep, ' best mae:', best_mae)
print('Epoch:', best_ndcg_ep, ' best ndcg:', best_ndcg)

Epoch: 99  best rmse: 0.9528607062081208
Epoch: 99  best mae: 0.7452099026218637
Epoch: 105  best ndcg: 0.8926667333490804


## Get the results

In [18]:
def precision(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)
    res = precision_score(y_true_binary, y_pred_binary)
    return res

def recall(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)
    res = recall_score(y_true_binary, y_pred_binary)
    return res

def f1(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)
    res = f1_score(y_true_binary, y_pred_binary)
    return res

def mae(y_true, y_pred):
    y_pred = np.clip(y_pred, 1., 5.)
    res = mean_absolute_error(y_true, y_pred)
    return res

def rmse(y_true, y_pred):
    y_pred = np.clip(y_pred, 1., 5.)
    res = mean_squared_error(y_true, y_pred)
    return np.sqrt(res)

In [21]:
best_model = CompleteNet(model, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale).double().to(device)
best_model.load_state_dict(torch.load('Glocal-K_best.pth'))
pred, _ = best_model(torch.Tensor(train_r).double().to(device))
pred = pred.float().cpu().detach().numpy()
y_pred = np.extract(test_m, pred)
y_true = np.extract(test_m, test_r)

best_mae = mae(y_true, y_pred)
best_rmse = rmse(y_true, y_pred)
best_precision = precision(y_true, y_pred)
best_recall = recall(y_true, y_pred)
best_f1 = f1(y_true, y_pred)

print('data: ', data_path)
print('best_mae: ', best_mae)
print('best_rmse: ', best_rmse)
print('best_precision: ', best_precision)
print('best_recall: ', best_recall)
print('best_f1: ', best_f1)

data:  ../processed_data_for_matrix_completion/discreted_link_ratings_p0.csv
best_mae:  0.8111106496160975
best_rmse:  1.0239966646299297
best_precision:  0.752067963335569
best_recall:  0.17457187337830826
best_f1:  0.2833677294360443
